# Credit Risk Modelling 


In [14]:
import mysql.connector
from mysql.connector import Error

def gettrainingdata():
    try:
        cnx = mysql.connector.connect(user='sql9284623', password='m8QTBgHjDr',
                                      host='sql9.freesqldatabase.com',
                                      database='sql9284623')
        cursor = cnx.cursor()
        query = "SELECT * FROM `CREDIT_DATA`"
        cursor.execute(query)
        fullrow = []
        partrow = []
        for row in cursor:
            fullrow.append(row)
            partrow.append(row[1])
        cursor.close()
        cnx.close()
        finalrow = [fullrow, partrow]
        return finalrow
    except Error as e:
        print("Error while connecting to MySQL database", e)

print(gettrainingdata())


[[(25, 35, 97, 3000, 5, 40, 10, 87), (89, 99, 56, 0, 1, 73, 3, 62), (42, 56, 74, 5000, 8, 57, 10, 55)], [35, 99, 56]]


In [56]:
# Defining tools 

import xgboost as xgboost
import pandas as pd
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

from xgboost import XGBRegressor
from sklearn.impute import SimpleImputer

# Loading data

accepted_data_path = "accepted_2007_to_2018Q3.csv"

accepted_data = pd.read_csv(accepted_data_path, nrows=20000, engine="python")

features = ["loan_amnt", "int_rate", "emp_length", "home_ownership", "annual_inc",
           "verification_status", "loan_status", "title", "delinq_2yrs", "fico_range_low", "fico_range_high", "mths_since_last_delinq", "mths_since_last_record",
            "pub_rec", "revol_bal", "revol_util", "application_type", "tot_cur_bal", "chargeoff_within_12_mths", "pub_rec_bankruptcies",
           "total_il_high_credit_limit"]

# Features that are not numbers:
# One Hot encoding: home_ownership, verification_status, title,  
# Direct change:
# - emp_length: a years (2 <= a <= 10, 10+ years, <1 year)  ===== done
# - loan_status: only takes Charged Off and Fully Paid  ===== done
# - application_type: deals only with individual   ====== done

# Direct change first:

chosen_data = accepted_data[features]
chosen_data.drop(chosen_data[(chosen_data.loan_status != "Fully Paid") & (chosen_data.loan_status != "Charged Off")].index, inplace = True)
chosen_data.drop(chosen_data[(chosen_data.application_type != "Individual")].index, inplace = True)


chosen_data = chosen_data.drop("application_type", axis = 1)


chosen_data["emp_length"].fillna(0, inplace = True)
emp_length_di = {"< 1 year": 0.5, "1 year": 1, "2 years": 2, "3 years": 3, "4 years": 4, "5 years": 5, "6 years": 6, "7 years": 7,
                "8 years": 8, "9 years": 9, "10+ years": 10}
chosen_data = chosen_data.replace({"emp_length": emp_length_di})

# One hot encoding:

X = chosen_data.drop("loan_status", axis = 1)

final_y = chosen_data.loan_status

final_X = pd.get_dummies(X)
# chosen_data.to_csv("updated.csv")

,loan_amnt,int_rate,emp_length,annual_inc,delinq_2yrs,fico_range_low,fico_range_high,mths_since_last_delinq,mths_since_last_record,pub_rec,...,title_Credit card refinancing,title_Debt consolidation,title_Green loan,title_Home buying,title_Home improvement,title_Major purchase,title_Medical expenses,title_Moving and relocation,title_Other,title_Vacation
2,12000.0,16.99,0.0,30000.0,0.0,665.0,669.0,NaN,72.0,3.0,...,1,0,0,0,0,0,0,0,0,0
7,25900.0,25.49,10.0,54000.0,0.0,685.0,689.0,56.0,NaN,0.0,...,1,0,0,0,0,0,0,0,0,0
9,8000.0,7.99,10.0,57000.0,3.0,680.0,684.0,7.0,NaN,0.0,...,1,0,0,0,0,0,0,0,0,0
12,13000.0,5.32,10.0,101800.0,0.0,720.0,724.0,NaN,79.0,3.0,...,1,0,0,0,0,0,0,0,0,0
16,5000.0,11.44,10.0,60000.0,0.0,665.0,669.0,NaN,NaN,0.0,...,1,0,0,0,0,0,0,0,0,0
17,25000.0,28.69,10.0,83807.4,0.0,690.0,694.0,39.0,NaN,0.0,...,0,1,0,0,0,0,0,0,0,0
19,4500.0,12.74,6.0,32000.0,0.0,660.0,664.0,NaN,NaN,0.0,...,1,0,0,0,0,0,0,0,0,0
22,11100.0,5.32,3.0,60000.0,0.0,725.0,729.0,NaN,NaN,0.0,...,1,0,0,0,0,0,0,0,0,0
23,12000.0,14.99,10.0,68000.0,0.0,670.0,674.0,26.0,NaN,0.0,...,0,1,0,0,0,0,0,0,0,0
25,20000.0,14.99,2.0,59200.0,1.0,660.0,664.0,0.0,NaN,0.0,...,0,0,0,0,0,1,0,0,0,0
